In [29]:
class HistoricalQuote:
    def __init__(
        self,
        base_currency: str,
        close: float,
        open: float,
        high: float,
        low: float,
        quote_currency: str,
    ):
        self.base_currency = base_currency
        self.close = close
        self.open = open
        self.high = high
        self.low = low
        self.quote_currency = quote_currency

    def __str__(self):
        return f"Historical Quote:\n\t\t\tBase Currency: {self.base_currency}\n\t\t\tQuote Currency: {self.quote_currency}\n\t\t\tOpen: {self.open}\n\t\t\tClose: {self.close}\n\t\t\tHigh: {self.high}\n\t\t\tLow: {self.low}"


class HistoricalResponse:
    def __init__(
        self, date: str, endpoint: str, request_time: str, quotes: list[HistoricalQuote]
    ):
        self.date = date
        self.endpoint = endpoint
        self.request_time = request_time
        self.quotes = quotes

    def __str__(self):
        quotes = ''
        for quote in self.quotes:
            quotes = quotes + str(quote)
        return f'Historical Response:\n\tDate: {self.date}\n\tRequest Time: {self.request_time}\n\tEndpoint: {self.endpoint}\n\tQuotes:\n\t\t{quotes}'

In [21]:
import requests


class MarketData:
    base_url = "https://marketdata.tradermade.com/api/v1/"
    historical_api = "historical?"
    live_api = "live?"
    date_parameter = "&date="
    currency_parameter = "&currency="
    api_key_parameter = "&api_key="

    def __init__(self, api_key):
        shared_parameters = self.api_key_parameter + api_key
        self.historical_url = self.base_url + self.historical_api + shared_parameters
        self.live_url = self.base_url + self.live_api + shared_parameters

    def live_data(self, currency_from, currency_to):
        url = self.live_url + self.currency_parameter + currency_from + currency_to
        print(url)

    def historical_data(self, currency_from, currency_to, date):
        url = (
            self.historical_url
            + self.date_parameter
            + date
            + self.currency_parameter
            + currency_from
            + currency_to
        )
        self.fetch_data(url)

    def fetch_data(self, url):
        response = requests.get(url)
        json_response = response.json()
        quotes_response = json_response.get("quotes")
        quotes: list[HistoricalQuote] = []
        for quote_response in quotes_response:
            quote: HistoricalQuote = HistoricalQuote(
                quote_response.get("base_currency"),
                quote_response.get("close"),
                quote_response.get("open"),
                quote_response.get("high"),
                quote_response.get("low"),
                quote_response.get("quote_currency"),
            )
            quotes.append(quote)

        historical_response: HistoricalResponse = HistoricalResponse(
            json_response.get("date"),
            json_response.get("endpoint"),
            json_response.get("request_time"),
            quotes,
        )
        print(historical_response)

    def __del__(self):
        self.live_url = ""
        self.historical_url = ""

In [30]:

from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('MARKETDATA_API_KEY', '')
metaData = MarketData(api_key)
metaData.historical_data("XAU", "THB", "2023-10-23")

Historical Response:
	Date: 2023-10-23
	Request Time: Fri, 27 Oct 2023 02:19:30 GMT
	Endpoint: historical
	Quotes:
		Historical Quote:
			Base Currency: XAU
			Quote Currency: THB
			Open: 71550.0
			Close: 71241.1765
			High: 71813.7255
			Low: 72605.0
